In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm
import pandas as pd
import re
import random
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
male_first_names = ['Harry', 'Mark', 'Daniel', 'Jeremy', 'Nathan', 'David', 'Robert', 'Tony', 'Jeffrey', 'Adam', 'Steve', 'John', 'Luke', 'Sam', 'Liam', 'Brian', 'Jason', 'Clark', 'Larry', 'Kevin', 'Oliver', 'William', 'Ronald', 'Henry', 'Nick', 'Richard', 'Matthew', 'Justin', 'Martin', 'Mike', 'Eric', 'Rick', 'Brandon', 'Tyler', 'Peter', 'Gebrial', 'Cam', 'Roy', 'Frank', 'Andrew', 'Patrick', 'Ethan', 'Roger', 'Jerry', 'Charlie', 'Donald', 'Michael', 'Ryan', 'Alan', 'Edward', 'George', 'Bob', 'Paul', 'Joshua', 'Gary', 'James', 'Thomas', 'Alex', 'Gabriel', 'Jake'] # 60
female_first_names = ['Emily', 'Susan', 'Lisa', 'Laura', 'Jane', 'Rachel', 'Emma', 'Sarah', 'Molly', 'Ashley', 'Teresa', 'Joan', 'Amelia', 'Olivia', 'Linda', 'Julia', 'Elizabeth', 'Mary', 'Karen', 'Betty', 'Amanda', 'Carol', 'Sandra', 'Nora', 'Pamela', 'Rebecca', 'Lucy', 'Carla', 'Helen', 'Diane', 'Janet', 'Gloria', 'Alice', 'Catherine', 'Maria', 'Anna', 'Kelly', 'Megan', 'Stephanie', 'Martha', 'Claire', 'Cameron', 'Eve', 'Samantha', 'Nicole', 'Barbara', 'Victoria', 'Nancy', 'Lauren', 'Lara', 'Anika', 'Jennifer', 'Jessica', 'Margaret', 'Melissa'] # 55
surnames = ['Wilson', 'Anderson', 'Thomson', 'Morrison', 'Clark', 'Moore', 'Parrish', 'Preston', 'Rogers', 'Coleman', 'Marlow', 'Webster', 'Hayden', 'Alison', 'Smith', 'Johnson', 'Williams', 'Brown', 'Jones', 'Collins', 'Cornell', 'Darcy', 'Deighton', 'Ellsworth', 'Hampton']

def substitute_surnames_with_titles(sentence, surnames_pool):
    def replace(match):
        title = match.group(1)
        current_surname = match.group(2)
        new_surname = random.choice([sn for sn in surnames_pool if sn != current_surname])
        return f"{title} {new_surname}"
    pattern = r'\b(Mr|Ms|Mister|Miss)\s+(\w+)'
    return re.sub(pattern, replace, sentence)

def generate_variations(original_sentence, male_names, female_names, all_surnames):
    variations = []

    for _ in range(4):
        used_names = set()
        surnames_pool = [surname for surname in all_surnames if surname not in used_names]

        sentence_variation = substitute_surnames_with_titles(original_sentence, surnames_pool) # no repetition within sentence
        used_names.update(surnames_pool)

        for name in male_names + female_names:
            if name in sentence_variation:
                if name in male_names:
                    available_male_names = [n for n in male_names if n not in used_names]
                    if available_male_names:
                        new_name = random.choice(available_male_names)
                        sentence_variation = sentence_variation.replace(name, new_name, 1)
                        used_names.add(new_name)
                else:
                    available_female_names = [n for n in female_names if n not in used_names]
                    if available_female_names:
                        new_name = random.choice(available_female_names)
                        sentence_variation = sentence_variation.replace(name, new_name, 1)
                        used_names.add(new_name)

        variations.append(sentence_variation)

    return variations

df = pd.read_csv('premises_v2.csv')
new_sentences = []
for premise in df['premise'].tolist():
    new_sentences += generate_variations(premise, male_first_names, female_first_names, surnames)
new_df = pd.DataFrame(new_sentences, columns=['premise'])
new_df.to_csv('1 extended_premises_names.csv', index=False)